<a href="https://colab.research.google.com/github/Bryan-Az/ContrastTransferLabs/blob/main/Supervised_Contrastive_Learning/SCL_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

github_path: Supervised_Contrastive_Learning/SCL_CV.ipynb